# 心電図からrr間隔を出力するプログラム

In [1]:
import sys
sys.path.append('..')
from Constants import Constants

In [2]:
import glob

In [3]:
import numpy as np
import pandas as pd
import heartpy as hp
from matplotlib import pyplot as plt

/Users/miyakooti/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
for i, subject in enumerate(Constants.subject_name_list):
    
    df_mean = pd.DataFrame()
    
    base_PLI_FC3 = 0
    base_PLI_FC4 = 0
    base_PLI_FCz = 0

    
    for j, phase in enumerate(Constants.phase_name_list):
        

        
        # PLI
        
        
        pathes = glob.glob(f"{Constants.path_to_matlab_repository_folder}/csv/{i}_{subject}/PLIdata/PLI_?_{subject}_{phase}.csv")
        path = pathes[0]
        
        
        save_path = f"{Constants.path_to_matlab_repository_folder}/csv/{i}_{subject}/PLIdata/mean_PLI_{subject}.csv"
        
        print("-------------------------------------------------")
        print(f"{subject}さんの{phase}フェーズ")
        df = pd.read_csv(path)

        
        columns_for_PLI = ['FpZ','FC3','FC4','FCz','O1','O2','Fp2']
        
        if phase == "rest":
            
            
            mean_values = []
            for column in columns_for_PLI:
                mean = df[column].mean()
                mean_values.append(mean)
                
                if column == "FC3":
                    base_PLI_FC3 = mean
                    
                if column == "FC4":
                    base_PLI_FC4 = mean
                    
                if column == "FCz":
                    base_PLI_FCz = mean

            # 初期化みたいな意味合い
            df_mean = pd.DataFrame([mean_values], columns=columns_for_PLI, index=[phase])

            # 2つのカラムを追加
            df_mean['mean_all'] = df_mean.mean(axis=1)
            mean_3ch = (df_mean['FC3'] + df_mean['FC4'] + df_mean['FCz']) / 3
            df_mean['mean_3ch'] = mean_3ch

            print(df_mean)
            
            
        else:
            
            mean_values = []
            concentration = []
            for column in columns_for_PLI:
                mean = df[column].mean()
                mean_values.append(mean)
                
                if column == "FC3":
                    concentration_FC3 = base_PLI_FC3 - mean
                    concentration.append(concentration_FC3)
                    
                if column == "FC4":
                    concentration_FC4 = base_PLI_FC4 - mean
                    concentration.append(concentration_FC4)
                    
                if column == "FCz":
                    concentration_FCz = base_PLI_FCz - mean
                    concentration.append(concentration_FCz)
            
            
            # 集中度。restからのPLI差分を計算することでいける
            
            row = np.concatenate([mean_values, concentration])
            column = np.concatenate([columns_for_PLI, ["concentration_FC3", "concentration_FC4", "concentration_FCz"]])
                

            df_for_add = pd.DataFrame([row], columns=column, index=[phase])
            
            concentration_mean = (df_for_add['concentration_FC3'] + df_for_add['concentration_FC4'] + df_for_add['concentration_FCz']) / 3
            df_for_add['concentration_mean'] = concentration_mean
            
            # /集中度

            
            # 2つのカラムを追加
            df_for_add['mean_all'] = df_for_add.mean(axis=1)
            mean_3ch = (df_for_add['FC3'] + df_for_add['FC4'] + df_for_add['FCz']) / 3
            df_for_add['mean_3ch'] = mean_3ch
            
            
            
            
            

            # append的な意味合い
            df_mean = pd.concat([df_mean, df_for_add])



        

    # 1subjectに対する保存処理をここのインデントで記載する
    print(df_mean)
    df_mean.to_csv(save_path)
        


        
#         print(df_mean)
            
            
            

        
        
        # Concentration
        
        # PLIから減算することで計算する
        
        
        
        


        
#         if phase_name == "rest":
                
                
            

#             df = pd.DataFrame(measures,index=[f"{phase_name}"])
#             df.to_csv(save_path, sep=',')
#         else:
            
#             #　差分を計算して辞書に追加
#             measures["lf_diff"] = lf_rest - measures["lf"]
#             measures["hf_diff"] = hf_rest - measures["hf"]
#             measures["lfhf_diff"] = lfhf_rest - measures["lf/hf"]
            
#             # mode="a"で上書きではなく追記モード、ヘッダーなし
#             df = pd.DataFrame(measures,index=[f"{phase_name}"])
#             df.to_csv(save_path, sep=',', mode="a", header=False)
#         hrdata = hp.get_data(path, delim=",", column_name="ECG")        

            
        # 卒論の図用スライス
        # hrdata = hrdata[int(15*Constants.fs):int(30*Constants.fs)
    
        # R波を検知している様子
        #hp.plotter(working_data, measures, figsize=(20,5))
        
#         rr_interval = pd.DataFrame(working_data["RR_list"])
#         rr_interval.to_csv(f"{Constants.path_to_matlab_repository_folder}/csv/{i}_{subject}/ECG/rr_interval/rr_interval_{subject}_{j}_{phase}.csv")

-------------------------------------------------
kumakuraさんのrestフェーズ
           FpZ       FC3       FC4       FCz        O1        O2       Fp2  \
rest  0.470375  0.718243  0.530592  0.783054  0.352428  0.286579  0.390179   

      mean_all  mean_3ch  
rest  0.504493  0.677296  
-------------------------------------------------
kumakuraさんのboredomフェーズ
-------------------------------------------------
kumakuraさんのflowフェーズ
-------------------------------------------------
kumakuraさんのultraフェーズ
-------------------------------------------------
kumakuraさんのoverloadフェーズ
               FpZ       FC3       FC4       FCz        O1        O2  \
rest      0.470375  0.718243  0.530592  0.783054  0.352428  0.286579   
boredom   0.276021  0.446082  0.437729  0.560950  0.180391  0.162976   
flow      0.262003  0.427572  0.435408  0.375186  0.661463  0.526907   
ultra     0.408234  0.557846  0.601885  0.547379  0.740779  0.643930   
overload  0.120266  0.291840  0.417778  0.511482  0.239870  0.206530   

-------------------------------------------------
moriさんのoverloadフェーズ
               FpZ       FC3       FC4       FCz        O1        O2  \
rest      0.413260  0.519220  0.549392  0.696971  0.218319  0.247750   
boredom   0.358975  0.495150  0.549294  0.599081  0.207888  0.253009   
flow      0.356516  0.343713  0.416962  0.506622  0.160450  0.167623   
ultra     0.283108  0.318627  0.425212  0.499669  0.207859  0.185039   
overload  0.338160  0.397541  0.378537  0.460065  0.263044  0.205037   

               Fp2  mean_all  mean_3ch  concentration_FC3  concentration_FC4  \
rest      0.270101  0.416431  0.588528                NaN                NaN   
boredom   0.326009  0.268377  0.547842           0.024070           0.000098   
flow      0.259813  0.261462  0.422433           0.175507           0.132430   
ultra     0.234323  0.259085  0.414503           0.200593           0.124181   
overload  0.264733  0.273913  0.412048           0.121679           0.170855   

          concen

IndexError: list index out of range